# Russian Synodal Bible (1885)

## Import

### Libraries

In [77]:
#import os
#from lxml import etree
from backend import *
import matplotlib
import string

In [2]:
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)
names_extractor = NamesExtractor(morph_vocab)

### Definitions

In [2]:
bibleTXT = './texts/bible/sinodalnyi-perevod.txt' 
booksJSON = './texts/bible/booksDict.json'
bibleJSON = './texts/bible/bible.json'
bibleIdJSON = './texts/bible/bibleID.json'
bibleXML = './texts/bible/bible.xml'
bibFrazyTXT = './texts/bible/Dubrovina_Slovar_Bibleyskikh_Frazeologizmov.txt'

bibleOHCO = ['test', 'book', 'chap', 'verse']
tokenCols = ['p_id', 'start', 'stop', 'text', 'token_id', 'head_id', 'rel', 'pos', 'lemma', 'anim', 'aspect', 'case', 'degree', 'gender', 'mood', 'number', 'person', 'tense', 'verb_form', 'voice']

chap_lines_re = '^===\s(\d{1,3})\s===$'
book_lines_re = '^==\s(.+)\s==$'
matt_name = 'От Матфея святое благовествование'

In [145]:
with open(booksJSON) as json_file: 
    booksDict = json.load(json_file)

## Pre-Processing

### Import Text into DF

In [135]:
with open(bibleTXT, 'r', encoding='windows-1251') as f: 
    bibleText = f.readlines()

bibliiaDf = pd.DataFrame(bibleText).rename(columns={0:'text'}).dropna()
bibliiaDf

,text
0,\n
1,== Бытие ==\n
2,\n
3,=== 1 ===\n
4,\n
...,...
41374,17 И Дух и невеста говорят: прииди! И слышавши...
41375,18 И я также свидетельствую всякому слышащему ...
41376,19 и если кто отнимет что от слов книги пророч...
41377,"20 Свидетельствующий сие говорит: ей, гряду ск..."


### Tidy Up

In [136]:
# remove blank lines and select puncutation, keeping only header & clean text lines
bibliiaDf.loc[:,'text'] = bibliiaDf.loc[:,'text'].str.replace(r'\n', '')
bibliiaDf.loc[:,'text'] = bibliiaDf.loc[:,'text'].str.replace('\]|\[|_|-|', '')
bibliiaDf = bibliiaDf.loc[bibliiaDf.text != '']
bibliiaDf

/tmp/ipykernel_242246/881465790.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  bibliiaDf.loc[:,'text'] = bibliiaDf.loc[:,'text'].str.replace(r'\n', '')
/tmp/ipykernel_242246/881465790.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  bibliiaDf.loc[:,'text'] = bibliiaDf.loc[:,'text'].str.replace('\]|\[|_|-|', '')


,text
1,== Бытие ==
3,=== 1 ===
5,1 В начале сотворил Бог небо и землю.
6,"2 Земля же была безвидна и пуста, и тьма над б..."
7,3 И сказал Бог: да будет свет. И стал свет.
...,...
41374,17 И Дух и невеста говорят: прииди! И слышавши...
41375,18 И я также свидетельствую всякому слышащему ...
41376,19 и если кто отнимет что от слов книги пророч...
41377,"20 Свидетельствующий сие говорит: ей, гряду ск..."


### Find Parts

In [141]:
# find df rows containing all the book and chapter title lines 
# as well as the OT/NT split on the 1st page of Matthew
chap_lines = bibliiaDf.loc[bibliiaDf.text.str.contains(chap_lines_re)].index
book_lines = bibliiaDf.loc[bibliiaDf.text.str.contains(book_lines_re)].index
test_line = bibliiaDf.loc[bibliiaDf.text.str.contains(matt_name)].index

# combine the title lines for extraction later
title_lines = test_line.append([chap_lines, book_lines]).sort_values()
title_lines

/tmp/ipykernel_242246/3255205770.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  chap_lines = bibliiaDf.loc[bibliiaDf.text.str.contains(chap_lines_re)].index
/tmp/ipykernel_242246/3255205770.py:4: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  book_lines = bibliiaDf.loc[bibliiaDf.text.str.contains(book_lines_re)].index


Int64Index([30299], dtype='int64', name='v_id')

### Assign OHCO Labels

In [142]:
# ['test']
bibliiaDf[bibleOHCO[0]] = np.where(bibliiaDf.index<test_line[0], 'O', 'N')
# ['book']
bibliiaDf[bibleOHCO[1]] = bibliiaDf.loc[book_lines].text.str.extract(book_lines_re)
# ['chap']
bibliiaDf[bibleOHCO[2]] = bibliiaDf.loc[chap_lines].text.str.extract(chap_lines_re)
# ['verse'] by splitting verse num at beginning from text in current verse lines
bibliiaDf[[bibleOHCO[3],'text']] = bibliiaDf.text.str.split(' ', 1, expand=True).iloc[:, [0, 1]]
# fill in book and chapter titles to cells below them
bibliiaDf[bibleOHCO[1:3]] = bibliiaDf[['book','chap']].ffill()
# drop title lines and reset index to give verse id num 
bibliiaDf = bibliiaDf.drop(title_lines, axis=0).reset_index()#.set_index(bibleOHCO)
bibliiaDf.index = range(1,len(bibliiaDf)+1)
bibliiaDf.index.name = 'v_id'
# reorder columns
bibliiaDf = bibliiaDf[['test', 'book', 'chap', 'verse', 'text']]
bibliiaDf

,test,book,chap,verse,text
v_id,,,,,
1,O,NaN,NaN,Бытие,==
2,O,NaN,NaN,1,===
3,O,NaN,NaN,В,начале сотворил Бог небо и землю.
4,O,NaN,NaN,Земля,"же была безвидна и пуста, и тьма над бездною, ..."
5,O,NaN,NaN,И,сказал Бог: да будет свет. И стал свет.
...,...,...,...,...,...
38543,N,NaN,NaN,И,Дух и невеста говорят: прииди! И слышавший да ...
38544,N,NaN,NaN,И,я также свидетельствую всякому слышащему слова...
38545,N,NaN,NaN,и,если кто отнимет что от слов книги пророчества...


### Swap Full Book Name for Abbreviation

In [143]:
book_nums = [(x+1,y) for x,y in enumerate(bibliiaDf.book.unique())]

In [ ]:
bibliiaDf.book = [booksDict[str(x)]['eng_abbr'] for x,y in book_nums for name in bibliiaDf.book if y == name]
bibliiaDf.sample(10)

### Export to JSON

## Processing

### OHCO DFs

#### Testaments

#### Books

#### Chapters

### Make XML

## Token DFs

In [153]:
TokenDf = pd.read_pickle('./proc/BibleVerseTokenDf.pkl')
TokenDf.set_index(['p_id', 'token_id'])

start stop      text head_id        rel    pos      lemma  \
p_id  token_id                                                             
1     1_1          0    1         В     1_2       case    ADP          в   
      1_2          2    8    начале     1_3        obl   NOUN     начало   
      1_3          9   17  сотворил     1_0       root   VERB  сотворить   
      1_4         18   21       Бог     1_3      nsubj  PROPN        бог   
      1_5         22   26      небо     1_3        obj   NOUN       небо   
...              ...  ...       ...     ...        ...    ...        ...   
37110 1_5         32   38    Христа     1_4  flat:name  PROPN    христос   
      1_6         39   41        со     1_8       case    ADP          с   
      1_7         42   47     всеми     1_8        det    DET       весь   
      1_8         48   52      вами     1_4       nmod   PRON         вы   
      2_1         54   59     Аминь     2_0       root  PROPN      аминь   

                anim aspect  case degree gender  mood number person tense  \
p_id  token_id                                                              
1     1_1       None   None  None   None   None  None   None   None  None   
      1_2       Inan   None   Loc   None   Neut  None   Sing   None  None   
      1_3       None   Perf  None   None   Masc   Ind   Sing   None  Past   
      1_4       Anim   None   Nom   None   Masc  None   Sing   None  None   
      1_5       Inan   None   Acc   None   Neut  None   Sing   None  None   
...              ...    ...   ...    ...    ...   ...    ...    ...   ...   
37110 1_5       Anim   None   Gen   None   Masc  None   Sing   None  None   
      1_6       None   None  None   None   None  None   None   None  None   
      1_7       None   None   Ins   None   None  None   Plur   None  None   
      1_8       None   None   Ins   None   None  None   Plur      2  None   
      2_1       Inan   None   Gen   None    Fem  None   Sing   None  None   

               verb_form voice  
p_id  token_id                  
1     1_1           None  None  
      1_2           None  None  
      1_3            Fin   Act  
      1_4           None  None  
      1_5           None  None  
...                  ...   ...  
37110 1_5           None  None  
      1_6           None  None  
      1_7           None  None  
      1_8           None  None  
      2_1           None  None  

[676266 rows x 18 columns]

In [167]:
bibliiaDf = pd.read_json(bibleJSON, orient='index')
bibliiaDf

,test,book,chap,verse,text
1,O,Gen,1,1,В начале сотворил Бог небо и землю.
2,O,Gen,1,2,"Земля же была безвидна и пуста, и тьма над без..."
3,O,Gen,1,3,И сказал Бог: да будет свет. И стал свет.
4,O,Gen,1,4,"И увидел Бог свет, что он хорош, и отделил Бог..."
5,O,Gen,1,5,"И назвал Бог свет днем, а тьму ночью. И был ве..."
...,...,...,...,...,...
37106,N,Rev,22,17,И Дух и невеста говорят: прииди! И слышавший д...
37107,N,Rev,22,18,И я также свидетельствую всякому слышащему сло...
37108,N,Rev,22,19,и если кто отнимет что от слов книги пророчест...
37109,N,Rev,22,20,"Свидетельствующий сие говорит: ей, гряду скоро..."


In [160]:
GetRankDf(TokenDf, 'lemma')#.plot(x='rank')

,n,rank
lemma,,
и,57470,1
он,19656,2
в,18350,3
они,12325,4
не,11883,5
...,...,...
выспросить,1,20980
рассудительные,1,20981
воздаст,1,20982


In [161]:
GetRankDf(TokenDf, 'pos')

,n,rank
pos,,
NOUN,147028,1
VERB,108063,2
PRON,78480,3
ADP,70754,4
CCONJ,63398,5
PROPN,55336,6
ADJ,35327,7
DET,34131,8
PART,23999,9


In [156]:
GetRankDf(TokenDf, 'rel')

,n,rank
rel,,
case,71010,1
nsubj,63995,2
cc,63839,3
conj,61390,4
obl,56899,5
obj,53924,6
nmod,44289,7
advmod,42893,8
root,42520,9


In [157]:
GetRankDf(TokenDf, 'anim')

,n,rank
anim,,
Inan,145289,1
Anim,83455,2


In [158]:
GetRankDf(TokenDf, 'person')

,n,rank
person,,
3,57935,1
2,22086,2
1,19806,3


In [159]:
VocabDf = TokenDf.loc[~TokenDf[tokenCols[9:]].isna().all(1)].lemma.value_counts().to_frame().reset_index()#.reset_index()
VocabDf.columns = ['lemma', 'count']
VocabDf.index = VocabDf.index.map(lambda x: x + 1).rename('rank')
VocabDf.iloc[:25]#.plot(x='rank')

,lemma,count
rank,,
1,он,19656
2,они,12325
3,быть,10839
4,я,10762
5,ты,7851
6,господь,7101
7,весь,6982
8,свой,6038
9,твой,5958


## Bible Phrase Dictionary

In [66]:
with open(bibFrazyTXT) as bibleFrazy: 
    bibleFrazyLines = bibleFrazy.readlines()

In [78]:
BibleFrazyDf = pd.DataFrame(bibleFrazyLines).rename(columns={0:'fraza'})
BibleFrazyDf.loc[:,'fraza'] = BibleFrazyDf.loc[:,'fraza'].apply(lambda x: x.strip().strip('\n').translate(str.maketrans('', '', string.punctuation)))
BibleFrazyDf = BibleFrazyDf.loc[BibleFrazyDf.fraza != '']

In [91]:
frazyList = BibleFrazyDf.loc[30189:].fraza.to_list()
FrazyDf = pd.DataFrame(data=frazyList).rename(columns={0:'fraza'})
FrazyDf.index.name = 'f_id'

In [ ]:
FrazyDf = nat_parse(FrazyDf, textCol='fraza')

In [85]:
caseWords = ['что', 'чего', 'чему', 'чём', 'чем',\
    'кто', 'кого', 'кем', 'ком', 'кому']

In [112]:
FrazyDf['caseword'] = [True if x in caseWords else False for x in FrazyDf['lemma']]

In [173]:
FrazyDf.set_index('p_id').loc[0]#.groupby('p_id', axis=1)

,start,stop,text,token_id,head_id,rel,pos,lemma,anim,aspect,case,degree,gender,mood,number,person,tense,verb_form,voice,caseword
p_id,,,,,,,,,,,,,,,,,,,,
0,0,5,Агнец,1_1,1_3,nsubj,PROPN,агнец,Anim,None,Nom,None,Masc,None,Sing,None,None,None,None,False
0,6,11,Божий,1_2,1_1,det,ADJ,божий,None,None,Nom,Pos,Masc,None,Sing,None,None,None,None,False
0,12,22,непорочный,1_3,1_0,root,VERB,непорочный,None,Perf,None,None,Masc,Ind,Sing,None,Past,Fin,Mid,False
0,23,26,как,1_4,1_10,mark,SCONJ,как,None,None,None,None,None,None,None,None,None,None,None,False
0,27,32,будто,1_5,1_6,fixed,PART,будто,None,None,None,None,None,None,None,None,None,None,None,False
0,33,36,как,1_6,1_10,mark,SCONJ,как,None,None,None,None,None,None,None,None,None,None,None,False
0,37,42,будто,1_7,1_6,fixed,PART,будто,None,None,None,None,None,None,None,None,None,None,None,False
0,43,49,словно,1_8,1_6,fixed,PART,словно,None,None,None,None,None,None,None,None,None,None,None,False
0,50,55,точно,1_9,1_10,advmod,ADV,точно,None,None,None,Pos,None,None,None,None,None,None,None,False


In [126]:
pos_lists = []
for x in range(0,(FrazyDf.p_id.max()+1)):
    pos_lists.append(FrazyDf.loc[FrazyDf.p_id == x].text.to_list())

In [128]:
FrazyAbbrsDf = BibleFrazyDf.iloc[:105].fraza.str.split(' – ', expand=True).rename(columns={0:"abbr", 1:"fraza"})
FrazyAbbrsDf.loc[:,'abbr'] = FrazyAbbrsDf.loc[:,'abbr'].apply(lambda x: x.strip())
FrazyAbbrsDf.loc[:,'fraza'] = FrazyAbbrsDf.loc[:,'fraza'].apply(lambda x: x.strip())
FrazyAbbrsDict = FrazyAbbrsDf.set_index('abbr').to_dict().get('fraza')